# 算法问题

## 文本最小编辑距离

In [2]:

def min_edit_distance(str1, str2):
    """
    求两个字符串间的最小编辑距离
    : param str1: string1
    : param str2: string2
    : return minimum edit-distance between str1 and str2 
    """
    len1 = len(str1)
    len2 = len(str2)
    
    # edit-distance matrix
    dp = [[ 0 for i in range(len2+1)] for j in range(len1+1)]
    
    # 
    for i in range(len1+1):
        dp[i][0] = i
    #
    for i in range(len2+1):
        dp[0][i] = i
           
    for i in range(1, len1+1):
        for j in range(1, len2+1):           
            # 增加操作
            insertion = dp[i][j-1] + 1;
            # 删除操作
            deletion = dp[i-1][j] + 1;
            # 替换操作
            replace = dp[i-1][j-1] + (1 if str1[i-1] != str2[j-1] else 0)
            # 三者取最小
            dp[i][j] = min(replace, min(insertion, deletion))
    
#     for i in range(len1+1):
#         for j in range(len2+1):
#             print(dp[i][j], end = ' ')
#         print()
    # min edit-distance     
    return dp[len1][len2]
    
    
"""
==test
"""  
if __name__ == "__main__":
    str1 = "hell"
    str2 = "hello"
    
    edit_dist = min_edit_distance(str1, str2)
    print(f'edit-distance:{edit_dist}')

edit-distance:1


## 寻找和最大的连续子数组

解决思路：
最大连续子串存在3种分布可能性：
   - 在数组的左半区
   - 在数组的右半区
   - 横跨数组的中点

这其中，需要特殊处理的，就是第三种情况，即处理横跨在数组中点的子数组:
   - 暴力寻找子数组在中点左侧的起始点（和最大）
   - 暴力寻找子数组在中点右侧的结束点（和最大）

第三种情况解决后，即可通过递归方式来寻找最终的子数组

In [28]:
import random

# 暴力检索整个解空间
# 时间复杂度为O(n^2)
def get_maxsum_subarray_ON2(array):
    
    array_len = len(array)
    max_subarray_sum = -1e6
    
    max_sub_start_idx = 0
    max_sub_end_idx = 0
    for i in range(array_len):
        t_sum = 0
        for j in range(i, array_len):
            t_sum += array[j]
            if t_sum > max_subarray_sum:
                max_subarray_sum = t_sum
                max_sub_start_idx = i
                max_sub_end_idx = j
            
    return max_subarray_sum, max_sub_start_idx, max_sub_end_idx     
    
# 分治策略 （divide-and-conquer）
# 时间复杂度为O(n*log2(n))
def get_maxsum_cross_subarray(array):
    """
    : param   array
    : return  maxsum, left_idx, right_idx
    """

    low = 0
    high = len(array)
    mid = int((low+high)/2)
    
    
    # get left cross subarray
    left_maxsum = -1e6
    left_idx = mid-1
    for i in range(low, mid):
        left_sum = sum(array[i:mid])
        if left_sum > left_maxsum:
            left_maxsum = left_sum
            left_idx = i
    
    
    # get right cross subarray
    right_maxsum = -1e6
    right_idx = mid
    for i in range(mid, high):
        right_sum = sum(array[mid:i+1])
        if right_sum > right_maxsum:
            right_maxsum = right_sum
            right_idx = i
            
    return left_maxsum+right_maxsum, left_idx, right_idx
    
def get_maxsum_subarray_recursion(array):
    """
    get max sum of subarray in array
    : param   array
    : return  maxsum, left_idx, right_idx
    """
    if len(array) == 1:
        return sum(array), 0, 0

    mid = int(len(array)/2)
    
    # get maxsum subarray in "left-half" of array
    lh_sum, lh_begin_idx, lh_end_idx = get_maxsum_subarray_recursion(array[:mid])
      
    # get maxsum subarray in "right-half" of array
    rh_sum, rh_begin_idx, rh_end_idx = get_maxsum_subarray_recursion(array[mid:])
                
    # get maxsum cross-mid-subarray in array
    cross_sum, cross_begin_idx, cross_end_idx = get_maxsum_cross_subarray(array)
    
    # return the subarray with the max sum
    if lh_sum >= rh_sum and lh_sum >= cross_sum:
        return lh_sum, lh_begin_idx, lh_end_idx
    elif rh_sum >= cross_sum:
        return rh_sum, rh_begin_idx, rh_end_idx
    else:
        return cross_sum, cross_begin_idx, cross_end_idx 

if __name__ == "__main__":
    
    array_list = [random.randint(-10, 20) for i in range(10)]
    print(array_list)

    max_sum, start_idx, end_idx = get_maxsum_subarray_ON2(array_list)
    
    print(f'first method: start @{start_idx+1}, end @{end_idx+1}, sum is {max_sum}')
    
    
    max_sum, start_idx, end_idx = get_maxsum_subarray_recursion(array_list)
    
    print(f'second method: start @{start_idx+1}, end @{end_idx+1}, sum is {max_sum}')

[-7, 17, 19, 5, -3, -6, -3, 13, -6, -1]
first method: start @2, end @8, sum is 42
second method: start @2, end @8, sum is 42


## 实现nms（非极大值拟制）

code: https://github.com/rbgirshick/py-faster-rcnn/lib/nms/py_cpu_nms.py

In [1]:
# --------------------------------------------------------
# Fast R-CNN
# Copyright (c) 2015 Microsoft
# Licensed under The MIT License [see LICENSE for details]
# Written by Ross Girshick
# --------------------------------------------------------

import numpy as np

def py_cpu_nms(dets, thresh):
    """Pure Python NMS baseline."""
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

In [24]:
import numpy as np
import random

test = np.random.randn(5)*20
print(test)


inds = np.where(test > 0)

print(inds)

# type(inds)


test2 = (1, 'test', [2, 3.2, 4])
print(test2)

type(test2)


[ -5.72543962   8.58523085   6.28998915  -2.33458522 -19.93379611]
(array([1, 2]),)
(1, 'test', [2, 3.2, 4])


tuple

## 解析四则运算表达式
 
 - 输入：四则运算表达式字符串<br/>
 - 输出：运算结果<br/>
 

### 方案1:

- 思路
    - 中缀表达式转后缀表达式
    - 后缀表达式计算结果
    
    
- 遗留问题:
    - ~~暂不考虑多位数, 数值范围:0~9~~  (2021.3.3 fix)
    - ~~暂不考虑浮点数~~  (2021.3.3 fix)
    - ~~暂不支持指数运算~~ (2021.3.3 fix)


In [101]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# @Time    : 2021/3/3 20:33
# @Author  : xujian
# @FileName: conv-ops.ipynb
# @Software: jupyter notebook
# @Github  :

import os
import math

class calculator:
    def __init__(self, exp_str=None):
        """
        Constructor of calculator
        :param exp_str (string) : operational expressions string
        """
        if not exp_str == None:
            self.exp_str = exp_str
        else:
            self.exp_str = []

    def setExp(self, exp_str):
        self.exp_str = exp_str

    def isNumber(self, str):
        """
        判断是否是数字（整型或浮点数）
        """
        try:
            float(str) # for int, long, float and complex
        except ValueError:
            if str != '.':
                return False
        return True

    def isOperator(self, str):
        """
        判断是否是运算符
        """
        if str in ['+', '-', '*', '/', '^']:
            return True
        return False

    def getPrecedence(self, op):
        """
        获取运算符的优先级
        """
        if op == '+' or op == '-':
            return 1
        elif op == '*' or op == '/':
            return 2
        elif op == '^':
            return 3
        raise Exception("invaild operator!")


    def execute(self, l_val, r_val, op):
        """
        获取单次运算结果
        """
        if op == '+':
            return l_val + r_val
        elif op == '-':
            return l_val - r_val
        elif op == '*':
            return l_val * r_val
        elif op == '/':
            return l_val / r_val
        elif op == '^':
            return math.pow(l_val, r_val)

    def calc(self, exp_str=None):
        """
        解析四则运算表达式,并返回结果
        """
        exp = []
        if exp_str == None:
            exp = self.exp_str
        else:
            exp = exp_str

        # operators & operands reserved stack(first in first out)
        ops_res = []

        # operators & operands to be executed queue(first in last out)
        ops_calc = []

        # operators stack
        result_stack = []

        # number_begin
        num_begin = -1
        num_end = -1

        for i in range(len(exp)):

            # 数字
            if self.isNumber(exp[i]):
                if num_begin < 0:
                    num_begin = i
                # last character
                num_end = i+1 if i == len(exp)-1 else -1

            else:
                num_end = i if num_begin >= 0 else -1

            # 数字直接push到 ops_calc队列
            if num_begin >= 0 and num_end >= 0:
                ops_calc.append(float(exp[num_begin:num_end]))
                num_begin, num_end = -1, -1

            # 左括号, push
            if exp[i] == '(':
                ops_res.append(exp[i])

            # 右括号, 栈内'('前的操作符&运算数全部pop
            if exp[i] == ')':
                # _ops_res = copy.deepcopy(ops_res)
                for op in reversed(ops_res):
                    if op == '(':
                        break
                    else:
                        ops_calc.append(op)
                        ops_res.pop()
                if ops_res[-1] == '(':
                    ops_res.pop()
                else:
                    raise Exception("mismatch brackets!")

            # 运算符
            if self.isOperator(exp[i]):
                if len(ops_res)>0 and self.isOperator(ops_res[-1]):
                    if self.getPrecedence(exp[i]) < self.getPrecedence(ops_res[-1]):
                        # 当前操作符比栈顶的操作符,优先级低
                        # 栈内操作符全部出栈
                        for op in reversed(ops_res):
                            if op == '(':
                                break
                            ops_calc.append(op)
                            ops_res.pop()


                # 当前操作符push入stack
                ops_res.append(exp[i])

        # 栈内操作符全部出栈
        for op in reversed(ops_res):
            ops_calc.append(op)
            ops_res.pop()

#         print(f'ops_calc is {ops_calc}')

        for op in ops_calc:

            # 数字, push入栈
            if self.isNumber(op):
                result_stack.append(op)

            # 运算符,从result_stack中提取前
            if self.isOperator(op):
                r_val = result_stack.pop()
                l_val = result_stack.pop()
                result = self.execute(l_val, r_val, op)
                result_stack.append(result)

#         print(len(result_stack))
#         print(result_stack[0])

        return(result_stack[0])

if __name__ == "__main__":

    # exp = '9+((3-1)*3-3)*2+10/2'
    exp = '9+((3-1)*3-3*(3+2))*2+10/2'
    
    # exp = '9+(3-1)*2+10/2'

    _calu = calculator();

    result = _calu.calc(exp) 

    print(f'result is {result}')


result is -4.0


### 方案2:

- 思路
    - 递归解决括号优先级问题
    - 创建2个list, 分别存储解析出来的操作符(operators)和运算数(operands)
    
    
- 遗留问题:    
    - 暂不支持指数运算
    

In [103]:
def isNumber(substr):
    """
    判断是否是数字（整型或浮点数）
    """
    try:
        float(substr) # for int, long, float and complex
    except ValueError:
        if substr != '.':
            return False
    return True
    
def isOperator(substr):
    """
    判断是否是运算符
    """
#     operator_list = ['+', '-', '*', '/']
    if substr in ['+', '-', '*', '/']:
        return True
    return False
    
def isBracket(substr):
    """
    判断是否是括号
    """
    if substr in ['(', ')']:
        return True
    return False


In [95]:
def calc(lvalue, rvalue, op):
    if op == '+':
        return lvalue+rvalue
    if op == '-':
        return lvalue-rvalue
    if op == '*':
        return lvalue*rvalue
    if op == '/':
        return lvalue/rvalue

    
def calculator(expression):
    """
    :param input: arithmetic expressions, including : 
                  - numbers: 
                      - integer: e.g. 10
                      - float: e.g. 1.5 
                  - operators: '+'、'-'、'*'、'/'
                  - bracket pairs：'('、')'
                  - etc.
                  sample: "10+3.5*2-4/(2+2)"
    @return calc result.
    """
    
    exp = expression
    
    # 运算符左边的操作数
    left_val = 0
    has_left_val = False
    # 运算符右边的操作数
    right_val = 0
    
    number_begin = -1
    number_end = -1
    
    # 运算符
    op = None
    
    if isOperator(exp[0]) or exp[0] == ')':
        raise Exception("expression begin with invalid operator!")
            
    bracket_left_idx = -1;
    
    for i in range(len(exp)):
        
        # 是否有左括号？有，跳过不处理
        if bracket_left_idx >= 0:
            continue
        
        
        # 空格，跳过不处理
        if isBlank(exp[i]):
            continue
                
        # 是否有左括号？
                
        # 左括号
        if exp[i] == '(':
            bracket_left_idx = i
            continue
                
        # 右括号      
        if exp[i] == ')':
            if bracket_left_idx>=0:
                # 优先计算括号内的运算
                bracket_result = calculator(exp[bracket_left_idx+1:i])
                if has_left_val:    # 有左运算数
                    if op is not None:
                        left_val = calc(left_val, bracket_result, op)
                    else:
                        raise Exception("expression brancket begin after nunber, not operator!")
                else:
                    left_val = bracket_result
            else:   
                raise Exception("expression mismatch ')'!")
               
        # 数字
        if isNumber(exp[i]):
            if number_begin == -1:
                nunber_begin = i
            else:
                number_end = i
        else:
            if number_begin >= 0:
                if isNumber(exp[number_begin:i]):
                    if has_left_val and op:
                        right_val = float(exp[number_begin:i])
                        left_val = calc(left_val, right_val, op)
                    else:
                        left_val = float(exp[number_begin:i])
                        has_left_val = True
                    nunber_begin = -1
                else:
                    raise Exception("expression invalid number!")
            
                
        # 运算符
        if isOperator(exp[i]):
            if not has_left_val:
                raise Exception("expression begin with invalid operator!")    
            op = exp[i] 
            if op == '+' or op == '-':
                right_val = calculator(exp[i+1:])
                left_val = calc(left_val, right_val, op)
                break 
            else:    # '*' or '/'
                continue
                
                
        raise Exception("expression has invaild character!")
    
    
        result = left_valie
    return result



if __name__ == "__main__":
    
    try:
        result = calculator("1+2*3") 
    except Exception as err:
        print(err)
    else:
        print(f'result is {result}')
    
    

expression has invaild character!


In [81]:
test1 = "hello world"
print(test1[2:])

llo world


# 排序问题

## 准备工作

参考：[常用的排序算法总结](https://zhuanlan.zhihu.com/p/40695917)

In [112]:
#coding=utf-8
from random import randint

# 生成随机序列
def generateRandomArray(n, min, max):
#     arr = []
    arr = [randint(min, max) for x in range(n)]
    return arr

# 生成基本有序的数列
def generateNearlyOrderedArray(n, swapTimes):
    arr = []
    for i in range(n):
        arr.append(i)
    
    for j in range(swapTimes):
        pos_x = randint(0, n-1)
        pos_y = randint(0, n-1)
        
        arr[pos_x], arr[pos_y] = arr[pos_y], arr[pos_x]
        
    return arr

# 判断序列是否有序
def isSorted(array):
    for i in range(0, len(array)-1):
        if array[i] > array[i+1]:
            return False
    return True

## 冒泡排序（bubble sort）

1. 时间复杂度

  $O(\frac{(n+1)n}{2})$ ,即 $O(n^2)$

2. 空间复杂度

   $O(1)$


In [37]:
def bubblesort(arr):
    n = len(arr)
    exchange = False
    for i in range(n-1, 0, -1):
        for j in range(0, i):
            if arr[j] > arr[j+1]:
                arr[j],arr[j+1] = arr[j+1],arr[j]
                exchange = True
        # 如果没有发生swap, 提前结束循环
        if exchange == False:
            break
    return arr

if __name__ == "__main__":
    
    array = generateRandomArray(10, 0, 20)
    print(f'原始序列：{array}')
    
    array = bubblesort(array)
    print(f'排序后：{array}')

原始序列：[5, 7, 12, 16, 11, 13, 20, 20, 17, 6]
排序后：[5, 6, 7, 11, 12, 13, 16, 17, 20, 20]


## 快速排序（quick sort）

1. 算法思想

    快速排序的基本思想：通过一趟排序将待排记录分隔成独立的两部分，其中一部分记录的关键字均比另一部分的关键字小，则可分别对这两部分记录继续进行排序，以达到整个序列有序。


2. 时间复杂度

- 理想情况下，需要拆分$log_2n$次，每次遍历n个数据
  时间复杂度 $O(n*\log_2n)$ ，

- 最坏情况下，原数组是逆序，拆分n次，分别遍历n，n-1, n-2, ..., 1次 时间复杂度为: $O(\frac{(n+1)n}{2})$ ,即 $O(n^2)$

- 平均时间复杂度: $O(n*\log_2n)$


3. 空间复杂度

   主要是递归造成的栈空间的使用，

- 理想情况，递归树的深度为: $\log_2n$, 其空间复杂度也就为 $O(\log_2n)$

- 最坏情况，需要进行n‐1递归调用，其空间复杂度为O(n)，

- 平均情况，空间复杂度也为:$O(\log_2n)$


4. 参考
- [图解排序算法及实现——快速排序 （Quick Sort）](https://blog.csdn.net/qq_20011607/article/details/82357239)
- [快速排序算法——C/C++](https://blog.csdn.net/weixin_42109012/article/details/91645051)

In [147]:
import random

def get_standard(array, low, high):
    """
    获取基准数据位置
    : param: array      数组
    : param: low        起始位置
    : param: high       结束位置
    : return standard pose
    """
    # 基准数据
    key = array[low]
    while(low < high):
        # 因为默认基准是从左边开始，所以从右边开始比较
        # 当队尾的元素大于等于基准数据 时,就一直向前挪动 high 指针
        while low < high and array[high] > key:
            high -= 1
            
        # 当找到比 array[low] 小的时，就把后面的值 array[high] 赋给它
        if low < high:
            array[low] = array[high]
        
        # 当队首元素小于等于基准数据 时,就一直向后挪动 i 指针
        while low < high and array[low] <= key: 
            low += 1
    
        # 当找到比 array[high] 大的时，就把前面的值 array[low] 赋给它
        if low < high:
            array[high] = array[low];
        
    # 跳出循环时 low 和 high 相等,此时的 low 或 high 就是 key 的正确索引位置
    # 把基准数据赋给正确位置
    array[low] = key
    return low

def adv_get_standard(array, low, high):
    """
    获取基准数据位置
    : param: array      数组
    : param: low        起始位置
    : param: high       结束位置
    : return standard pose
    """
    
    # 随机选择基准数据，减少最极端情况下带来的数据不均衡问题
    mid = random.randint(low, high) 
    array[mid], array[low] = array[low], array[mid]
    

    # 基准数据
    mid = low
    key = array[low]
            
    while(low < high):
        # 因为默认基准是从左边开始，所以从右边开始比较
        # 当队尾的元素大于等于基准数据 时,就一直向前挪动 high 指针
        if array[high] >= key:
            high -= 1
            continue
            
        # 当队首元素小于等于基准数据 时,就一直向后挪动 low 指针
        if array[low] <= key: 
            low += 1
            continue
            
        # 当找到比 array[low] 小的时，就把后面的值 array[high] 赋给它
        # 当找到比 array[high] 大的时，就把前面的值 array[low] 赋给它
        if array[high] < key and array[low] > key:
            array[high], array[low] = array[low] , array[high]
            
#             print('swap', " low: ", low, array[low], " high: ",high, array[high])

        
    # 跳出循环时 low 和 high 相等,此时的 low 或 high 就是 key 的正确索引位置
    # 把基准数据赋给正确位置
    # print('mid-swap', "low:", low, array[low], "mid:", mid, array[mid])
    array[mid], array[low] = array[low], array[mid]
    mid = low
    
    return mid



def quicksort(array, low, high):
    
    # 开始默认基准为 low
    if low < high:         
        # 分段位置下标
        standard = adv_get_standard(array, low, high);
        # 递归调用排序
        # 左边排序
        quicksort(array, low, standard - 1);
        # 右边排序
        quicksort(array, standard + 1, high);
    
    return array



if __name__ == "__main__":
    
    # 生成一个10个元素的随机list
    array_list = [random.randint(1, 100) for i in range(10)]
    
    array_list = [32, 7, 34, 17, 90, 27, 94, 55, 70, 26]
    
    print(f'排序前: {array_list}')
    
    # 快速排序
    quicksort(array_list, 0, len(array_list)-1)
    
    print(f'排序后: {array_list}')
    
    
    if isSorted(array_list):
        print("排序结果正确！")
    else:
        print("排序结果错误")

排序前: [32, 7, 34, 17, 90, 27, 94, 55, 70, 26]
排序后: [7, 17, 26, 27, 32, 34, 55, 70, 90, 94]
排序结果正确！


## 归并排序（merge sort）

分治思想: divide-and-conquer

1. 时间复杂度
    拆分$log_2n$次，每次遍历n个数据
    
- 理想情况下，时间复杂度 $O(n*\log_2n)$，如果改进的算法，在有序数组上，不需要merge，时间复杂度降为：$O(n)$

- 最坏情况下，时间复杂度 $O(n*\log_2n)$

- 平均时间复杂度: 时间复杂度 $O(n*\log_2n)$

2. 空间复杂度

   主要是临时数据和递归造成的栈空间的使用，$O(n+\log_2n)$, 即 $O(n)$

In [25]:
def _merge(array, low, mid, high):
    """
    合并array左右两个子数组 array[r:mid] array[mid+1:high]
    : param array:   数组
    : param low  :   array的最左侧下标
    : param high :   array的最右侧下标
    : param mid  :   array的中间下标
    : return None 
    """
    # 额外开辟空间, 拷贝arr[l,..,r]到_array数组，然后在原array上赋值操作
    _array = array[low:high+1]
    
    # 分别从左边开始scan
    i, j = low, mid+1 
    
    for k in range(low, high+1):
        
        # 左子串边界判断
        if i > mid:          # 左子串已经merge完
            array[k] = _array[j-low]
            j += 1
        elif j > high:       # 右子串已经merge完
            array[k] = _array[i-low]
            i += 1
        elif _array[i-low] < _array[j-low]:
            array[k] = _array[i-low]
            i += 1
        else:
            array[k] = _array[j-low]
            j += 1

def merge_sort(array, low, high):
    """
    递归归并排序array
    : param array:   数组
    : param low  :   array的最左侧下标
    : param high :   array的最右侧下标
    : return None 
    """
    if low >= high:
        return 
    mid = low + (high-low)//2        # (r+high)//2 的改进算法，防止(r+high)太大溢出
    merge_sort(array, low, mid)      # 对左半边array 归并排序
    merge_sort(array, mid+1, high)   # 对右半边array 归并排序
    
    # 只有对于arr[mid] <= arr[mid+1]的情况,无需merge
    # 对于近乎有序的数组非常有效,但是对于一般情况,有一定的性能损失
    if array[mid] > array[mid+1]:
        _merge(array, low, mid, high)    # 合并


def merge_sort_bottomup(array, low, high):
    """
    递归归并排序array
    : param array:   数组
    : param low  :   array的最左侧下标
    : param high :   array的最右侧下标
    : return None 
    """
    
    # 逐层迭代，从最底层开始，sz = low → high 
    step = 1
    while step <= (high-low+1):
        # 每层，从左往右两两归并，i = 0 → n-1
        i = low
        while i<=high:
            # 对 arr[i...i+sz-1] 和 arr[i+sz...i+2*sz-1] 进行归并
            _merge(array, i, i+step-1, min(i+step*2-1,high))
            i += step*2 # 跳到下一对
        step += step
        

import random
if __name__ == "__main__":
    
    # 生成一个10个元素的随机list
    array_list = [random.randint(1, 100) for i in range(10)]
    
    print(f'排序前: {array_list}')
    
    # 快速排序
    merge_sort_bottomup(array_list, 0, len(array_list)-1)
    
    print(f'排序后: {array_list}')

    

排序前: [34, 82, 33, 100, 99, 2, 89, 28, 58, 91]
排序后: [2, 28, 33, 34, 58, 82, 89, 91, 99, 100]


## 堆排序（heap sort）

---
1. 堆的定义
堆是具有以下性质的完全二叉树：
 - 每个结点的值都大于或等于其左右孩子结点的值，称为大顶堆；
 - 每个结点的值都小于或等于其左右孩子结点的值，称为小顶堆。
 
如下图：
![image](https://images2015.cnblogs.com/blog/1024555/201612/1024555-20161217182750011-675658660.png)

同时，我们对堆中的结点按层进行编号，将这种逻辑结构映射到数组中就是下面这个样子

![image](https://images2015.cnblogs.com/blog/1024555/201612/1024555-20161217182857323-2092264199.png)

该数组从逻辑上讲就是一个堆结构，我们用简单的公式来描述一下堆的定义就是：

大顶堆：arr[i] >= arr[2i+1] && arr[i] >= arr[2i+2] 

小顶堆：arr[i] <= arr[2i+1] && arr[i] <= arr[2i+2] 

---
2. 基本思路：

　　a.将无需序列构建成一个堆，根据升序降序需求选择大顶堆或小顶堆;<br/>
　　b.将堆顶元素与末尾元素交换，将最大元素"沉"到数组末端;<br/>
　　c.重新调整结构（从左至右，从下至上进行调整），使其满足堆定义，然后继续交换堆顶元素与当前末尾元素，反复执行调整+交换步骤，直到整个序列有序。<br/>

---
3. 时间复杂度

- 初始化堆，时间复杂度为 $O(n)$ ，

- 交换并重建堆的过程中，需交换n-1次，时间复杂度为: $O(\log_2(n-1)+\log_2(n-2)+....+1)$ ,近似为 $O(n*\log_2n)$

- 综上复杂度: $O(n*\log_2n)$

---
4. 空间复杂度

   堆排序是就地排序，空间复杂度为常数：$O(1)$



[图解排序算法(三)之堆排序](https://www.cnblogs.com/chengxiao/p/6129630.html)

In [44]:
def heap_shift(arr, start, end):
    """最大堆调整"""
    root = start
    while True:
        child = 2 * root + 1     # left child
        if child > end:
            break
        if child + 1 <= end and arr[child] < arr[child + 1]:
            child += 1               # right child

        if arr[child] > arr[root]:
            arr[child], arr[root] = arr[root], arr[child]
            root = child
        else:
            break
        

def heap_sort(array):
    # init max-heap
    for start in range(len(array)//2-1, -1, -1):
        heap_shift(array, start, len(array)-1)
        
    # heap sort
    for end in range(len(array)-1, 0, -1):
        array[0], array[end] = array[end], array[0]
        heap_shift(array, 0, end-1)
    
    return array

if __name__ == "__main__":
    
    # 生成一个10个元素的随机list
    array_list = [random.randint(1, 100) for i in range(10)]
    
    print(f'排序前: {array_list}')
    
    # 堆排序
    array_list = heap_sort(array_list)
    
    
    
    print(f'排序后: {array_list}')

排序前: [95, 70, 34, 72, 52, 45, 8, 97, 69, 84]
排序后: [8, 34, 45, 52, 69, 70, 72, 84, 95, 97]
